In [1]:
import sys
sys.path.append('scripts')
from lib import *
from sklearn.metrics import confusion_matrix, classification_report
from math import ceil
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [2]:
dfs, available_classes, available_models = read_raw_reports(os.path.join('..',raw_reports_path))

# replacing spaces with underscores:
available_classes = [cname.replace(' ', '_') for cname in available_classes]

In [3]:
# having it reverse for plot:
available_classes_r = available_classes[::-1]

available_classes_r_upper = [cname.upper() for cname in available_classes_r]

In [4]:
available_models

['ViT-B_32',
 'ViT-L_14',
 'ViT-bigG-14-CLIPA-336',
 'EVA02-E-14-plus',
 'ViT-H-14-378-quickgelu',
 'ViT-SO400M-14-SigLIP-384',
 'RN101',
 'RN50x64']

In [5]:
available_classes

['asphalt',
 'concrete',
 'concrete_plates',
 'grass',
 'ground',
 'paving_stones',
 'sett']

In [6]:
all_dfs = []
modelwise_dfs = {}

for class_dict in dfs:
    for model_name in dfs[class_dict]:
        model_df = dfs[class_dict][model_name]
        all_dfs.append(model_df)

        if not model_name in modelwise_dfs:
            modelwise_dfs[model_name] = []

        modelwise_dfs[model_name].append(model_df)


for model_name in modelwise_dfs:
    modelwise_dfs[model_name] = pd.concat(modelwise_dfs[model_name]).copy()

In [7]:
{model_name: len(modelwise_dfs[model_name]) for model_name in modelwise_dfs}

{'EVA02-E-14-plus': 2619,
 'RN101': 2619,
 'RN50x64': 2619,
 'ViT-bigG-14-CLIPA-336': 2619,
 'ViT-B_32': 2619,
 'ViT-H-14-378-quickgelu': 2619,
 'ViT-L_14': 2619,
 'ViT-SO400M-14-SigLIP-384': 2619}

In [8]:
merged_df = pd.concat(all_dfs)

In [9]:
true_classes = merged_df.groupby('image_name').agg('first')[class_label] #.str.replace('_', ' ')
merged_pred = merged_df.groupby('image_name').sum(numeric_only=True)

merged_pred[prediction_label] = merged_pred.idxmax(axis=1)
merged_pred[class_label] = true_classes


In [10]:
merged_pred

,asphalt,concrete,concrete_plates,grass,ground,paving_stones,sett,prediction,class
image_name,,,,,,,,,
00098db7c6cd6ee36d590ff0d5db7ad1.png,0.213754,3.985507,0.961183,0.019217,0.534829,1.280490,0.005109,concrete,concrete
00157343508e08f1dbc0efc1f8066f3d.png,0.601964,0.216773,0.454248,0.994805,2.932286,1.649854,0.149998,ground,ground
003a08eff81fe589f95102bf2ce1ecf5.png,0.007631,0.002122,0.013925,5.955048,0.965288,0.043469,0.012785,grass,grass
005b94f55127ae460c0741eac3a89b25.png,4.938634,0.277412,1.465858,0.012804,0.181552,0.108702,0.015524,asphalt,asphalt
006d04cdfa79fac8f108a0a1cd34d0f8.png,0.018306,0.004128,0.016005,6.087209,0.791766,0.057336,0.025230,grass,grass
...,...,...,...,...,...,...,...,...,...
ffbd835981460b0e1b7825681c7b9449.png,0.544958,0.495025,0.475408,0.098204,0.890578,3.950697,0.545128,paving_stones,concrete_plates
ffd0ab8395fc05d8d14f3cfe8a6a6b4a.png,0.600495,0.556082,0.496194,0.633679,3.802484,0.742776,0.168145,ground,ground
ffd4bc27fe750ca5fedac61da4f7a397.png,0.428585,0.148436,0.752849,0.359114,3.330605,1.677660,0.302741,ground,ground


In [11]:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,
                         predicted_labels,
                         labels=available_classes_r,
                          #  normalize='all'
                           ) #* 100
c_mat = c_mat.round(3)

In [12]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [13]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes_r,
    output_dict=True
)

{'sett': {'precision': 0.0, 'recall': 0.0, 'f1-score': 0.0, 'support': 346.0},
 'paving_stones': {'precision': 0.4550561797752809,
  'recall': 0.9788519637462235,
  'f1-score': 0.6212847555129435,
  'support': 331.0},
 'ground': {'precision': 0.7236024844720497,
  'recall': 0.9872881355932204,
  'f1-score': 0.8351254480286738,
  'support': 472.0},
 'grass': {'precision': 0.9166666666666666,
  'recall': 0.9890350877192983,
  'f1-score': 0.9514767932489452,
  'support': 456.0},
 'concrete_plates': {'precision': 0.46551724137931033,
  'recall': 0.15428571428571428,
  'f1-score': 0.2317596566523605,
  'support': 175.0},
 'concrete': {'precision': 0.6896551724137931,
  'recall': 0.7253886010362695,
  'f1-score': 0.7070707070707071,
  'support': 386.0},
 'asphalt': {'precision': 0.839041095890411,
  'recall': 0.5408388520971302,
  'f1-score': 0.6577181208053692,
  'support': 453.0},
 'accuracy': 0.6846124474990455,
 'macro avg': {'precision': 0.584219834371073,
  'recall': 0.6250983363539795

In [14]:
n_rows = int(ceil(len(available_models)/2))

In [15]:
modelwise_confusion_matrices = {}

for model in modelwise_dfs:
    true_labels =      modelwise_dfs[model][class_label]#.tolist()
    predicted_labels = modelwise_dfs[model][prediction_label]#.tolist()

    modelwise_confusion_matrices[model] = confusion_matrix(
        true_labels,
        predicted_labels,
        labels=available_classes,
        # normalize='true'
        ) #*  100
    modelwise_confusion_matrices[model] = modelwise_confusion_matrices[model].round(3)

In [16]:
modelwise_confusion_matrices

{'EVA02-E-14-plus': array([[270, 128,   4,  23,  17,   0,  11],
        [ 66, 280,   3,  12,  21,   1,   3],
        [ 17,  56,   8,  14,  43,  37,   0],
        [  2,   1,   0, 449,   4,   0,   0],
        [ 71,   9,   0,   7, 367,   4,  14],
        [ 16,   3,   0,   0,   7, 305,   0],
        [ 14,  11,   0,   6,  41, 274,   0]], dtype=int64),
 'RN101': array([[108,   1, 172,  21,  88,  23,  40],
        [ 10,   3, 250,   1,  22,  78,  22],
        [  1,   1,  88,   1,  18,  66,   0],
        [  0,   0,   4, 384,  65,   1,   2],
        [  2,   0,  26,   7, 360,  75,   2],
        [  2,   0,  12,   0,   2, 314,   1],
        [  8,   0,  20,   2,  14, 302,   0]], dtype=int64),
 'RN50x64': array([[ 45, 206,   1, 124,  12,   6,  59],
        [  3, 196,   0,  37,  12,  49,  89],
        [  5,  46,   0,  22,  33,  66,   3],
        [  0,   0,   0, 402,  48,   4,   2],
        [ 78,  84,   0, 149, 127,  16,  18],
        [  8,  21,   0,   1,   5, 296,   0],
        [  9,  29,   0,  13,  4

In [17]:
# fig = go.Figure().set_subplots(rows=n_rows, cols=2, shared_xaxes=True, shared_yaxes=True, vertical_spacing=0.05, horizontal_spacing=0.01)
fig = make_subplots(rows=n_rows, cols=2, 
                    shared_xaxes=True, 
                    shared_yaxes=True,
                    subplot_titles=list(modelwise_confusion_matrices.keys()),
                    vertical_spacing=0.015, 
                    horizontal_spacing=0.01,
                    # x_title="Prediction",
                    # y_title="Class",
                    )

for i,modelname in enumerate(modelwise_confusion_matrices):
    subplot = px.imshow(modelwise_confusion_matrices[modelname],
                
                x=available_classes,
                y=available_classes,
              text_auto=True, 
              #  aspect="auto",
               )
    
    # print(i//2+1,i%2+1)
    subplot['layout']['xaxis']['autorange'] = "reversed"

    fig.add_trace(subplot.data[0], row=i//2+1, col=i%2+1)
    fig.update_xaxes(row=i//2+1, col=i%2+1, autorange='reversed')
    # fig.append_trace(subplot.data[0], row=i//2+1, col=i%2+1)

fig.update_coloraxes(colorscale=px.colors.sequential.Burg)
fig.update_layout(height=400*n_rows, width=400*2,
                  xaxis_title="Prediction",
                  yaxis_title="Class",
                  showlegend=True,)
fig.show()


In [18]:
# getting the diagonal sum of the confusion matrix:
diagonals = {}

for model in modelwise_confusion_matrices:
    diagonals[model] = modelwise_confusion_matrices[model].diagonal().sum()

diagonals['combined'] = c_mat.diagonal().sum() 

diagonals_df = pd.DataFrame.from_dict(diagonals, orient='index',columns=['score']).reset_index().rename(columns={'index':'model_name'}).sort_values(by='score', ascending=False)

diagonals_df['score'] = round(100 *  diagonals_df['score'] / (len(available_models) * 100),2)

diagonals_df

,model_name,score
8,combined,224.12
0,EVA02-E-14-plus,209.88
3,ViT-bigG-14-CLIPA-336,209.75
5,ViT-H-14-378-quickgelu,209.25
7,ViT-SO400M-14-SigLIP-384,206.25
6,ViT-L_14,178.12
1,RN101,157.12
4,ViT-B_32,143.62
2,RN50x64,133.25


In [19]:
fig = px.bar(diagonals_df,x='model_name', y='score', text_auto=True)
fig.show()


In [20]:
selected_models = {}

for modelname in ['ViT-H-14-378-quickgelu', 'ViT-SO400M-14-SigLIP-384', 'EVA02-E-14-plus']:
    selected_models[modelname] = modelwise_dfs[modelname]

In [21]:
merged_selected = pd.concat(selected_models).reset_index()

In [22]:
true_classes = merged_selected.groupby('image_name').agg('first')[class_label] #.str.replace('_', ' ') #.reset_index().rename(columns={class_label:'class'})

In [23]:
merged_selected = merged_selected.groupby('image_name').sum(numeric_only=True)

In [24]:
merged_selected.drop(columns=['level_1'], inplace=True)

In [25]:
merged_pred[prediction_label] = merged_selected.idxmax(axis=1)
merged_pred[class_label] = true_classes.tolist()

In [26]:
# computing the confusion matrix:
true_labels =      merged_pred[class_label]#.tolist()
predicted_labels = merged_pred[prediction_label]#.tolist()

c_mat = confusion_matrix(true_labels,predicted_labels,labels=available_classes_r,
                        #   normalize='true'
                          ) #* 100
c_mat = c_mat.round(3)

In [27]:
fig = px.imshow(c_mat,
                labels=dict(x="Prediction", y="Class"),
                x=available_classes_r,
                y=available_classes_r,
                text_auto=True, 
            #    aspect="auto",
                color_continuous_scale=px.colors.sequential.Burg,
                width=700,
                height=700,
               )
fig.update_xaxes(side="top")

# fig['layout']['xaxis']['autorange'] = "reversed"

fig.show()

In [28]:
classification_report(
    true_labels,
    predicted_labels,
    labels=available_classes_r,
    output_dict=True
)

{'sett': {'precision': 0.46153846153846156,
  'recall': 0.017341040462427744,
  'f1-score': 0.033426183844011144,
  'support': 346.0},
 'paving_stones': {'precision': 0.4960380348652932,
  'recall': 0.945619335347432,
  'f1-score': 0.6507276507276507,
  'support': 331.0},
 'ground': {'precision': 0.7444444444444445,
  'recall': 0.9936440677966102,
  'f1-score': 0.8511796733212341,
  'support': 472.0},
 'grass': {'precision': 0.9356846473029046,
  'recall': 0.9890350877192983,
  'f1-score': 0.9616204690831557,
  'support': 456.0},
 'concrete_plates': {'precision': 0.7358490566037735,
  'recall': 0.22285714285714286,
  'f1-score': 0.34210526315789475,
  'support': 175.0},
 'concrete': {'precision': 0.6531049250535332,
  'recall': 0.7901554404145078,
  'f1-score': 0.7151230949589683,
  'support': 386.0},
 'asphalt': {'precision': 0.7784256559766763,
  'recall': 0.5894039735099338,
  'f1-score': 0.6708542713567839,
  'support': 453.0},
 'accuracy': 0.7063764795723558,
 'macro avg': {'preci

In [29]:
100 * (c_mat.diagonal().sum() / (100 * len(available_models)))

231.25

In [30]:
from sklearn.preprocessing import LabelBinarizer
import matplotlib.pyplot as plt
from sklearn.metrics import RocCurveDisplay
import numpy as np

In [31]:
label_binarizer = LabelBinarizer().fit(predicted_labels)


In [32]:
y_onehot_test = label_binarizer.transform(true_labels)

In [33]:
label_binarizer.transform(["concrete"])

array([[0, 1, 0, 0, 0, 0, 0]])

In [34]:
class_of_interest = "concrete"
class_id = np.flatnonzero(label_binarizer.classes_ == class_of_interest)[0]
class_id

1

In [35]:
y_score = np.array(merged_df.select_dtypes(include=['float64']))

In [36]:
display = RocCurveDisplay.from_predictions(
    y_onehot_test[:, class_id],
    y_score[:, class_id],
    name=f"{class_of_interest} vs the rest",
    plot_chance_level=True,
)
_ = display.ax_.set(
    xlabel="False Positive Rate",
    ylabel="True Positive Rate",
    title="One-vs-Rest ROC curves",
)

ValueError: Found input variables with inconsistent numbers of samples: [2619, 20952]

THE SAMPLE CODE AT:

https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html

In [45]:
import numpy as np

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
target_names = iris.target_names
X, y = iris.data, iris.target
y = iris.target_names[y]

random_state = np.random.RandomState(0)
n_samples, n_features = X.shape
n_classes = len(np.unique(y))
X = np.concatenate([X, random_state.randn(n_samples, 200 * n_features)], axis=1)
(
    X_train,
    X_test,
    y_train,
    y_test,
) = train_test_split(X, y, test_size=0.5, stratify=y, random_state=0)

In [46]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
y_score = classifier.fit(X_train, y_train).predict_proba(X_test)